In [1]:
import trax

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


In [2]:
trax

<module 'trax' from '/usr/local/lib/python3.7/site-packages/trax/__init__.py'>

In [5]:
trax.models.reformer.Reformer??

In [7]:
trax.models.TransformerEncoder(n_)

In [8]:
df = pd.read_csv("class1_sequences.csv")

In [11]:
from pepdata.amino_acid_alphabet import peptide_to_indices
encoded_sequences = df.seq.str.replace("?", "X").map(peptide_to_indices).values

In [12]:
encoded_sequences

array([list([12, 0, 19, 12, 0, 14, 1, 16, 10, 10, 10, 10, 10, 15, 7, 0, 10, 0, 10, 16, 6, 16, 17, 0, 7, 15, 8, 15, 12, 1, 18, 13, 13, 16, 15, 19, 15, 1, 14, 7, 1, 7, 5, 14, 1, 13, 9, 0, 19, 7, 18, 19, 3, 3, 16, 6, 13, 19, 1, 13, 3, 15, 3, 0, 0, 15, 6, 11, 12, 5, 14, 1, 0, 14, 17, 9, 5, 6, 5, 7, 14, 5, 18, 17, 3, 6, 5, 16, 1, 2, 12, 11, 0, 8, 15, 6, 16, 3, 1, 0, 2, 10, 7, 16, 10, 1, 7, 18, 18, 2, 6, 15, 5, 3, 7, 15, 8, 16, 9, 6, 9, 12, 18, 7, 4, 3, 19, 7, 14, 3, 7, 1, 13, 10, 1, 7, 18, 1, 6, 3, 0, 18, 3, 7, 11, 3, 18, 9, 0, 10, 2, 5, 3, 10, 1, 15, 17, 16, 0, 0, 3, 12, 0, 0, 6, 9, 16, 11, 1, 11, 17, 5, 0, 19, 8, 0, 0, 5, 6, 1, 1, 19, 18, 10, 5, 7, 1, 4, 19, 3, 7, 10, 1, 1, 18, 10, 5, 2, 7, 11, 5, 16, 10, 6, 1, 16, 3, 14, 14, 11, 16, 8, 12, 16, 8, 8, 14, 9, 15, 3, 8, 5, 0, 16, 10, 1, 4, 17, 0, 10, 7, 13, 18, 14, 0, 5, 9, 16, 10, 16, 17, 6, 1, 3, 7, 5, 3, 6, 16, 6, 3, 16, 5, 10, 19, 5, 16, 1, 14, 0, 7, 3, 7, 16, 13, 6, 11, 17, 0, 0, 19, 19, 19, 14, 15, 7, 5, 5, 6, 1, 18, 16, 4, 8, 19, 6, 8

In [27]:
unique_species = set(df.species_prefix.unique())
unique_species_list = list(unique_species)
unique_species_indices = {s:i for i, s in enumerate(unique_species_list)}
species_indices = np.array([unique_species_indices[s] for s in df.species_prefix], dtype='int32')
n_species = len(unique_species)

In [57]:
transformer = trax.models.TransformerEncoder(
    vocab_size=22,
    n_classes=n_species,
    d_model=512,
    d_ff=2048,
    n_layers=2,
    n_heads=8,
    dropout=0.1,
    dropout_shared_axes=None,
    max_len=1024,
)

In [74]:

model = tl.Serial(
    tl.Embedding(22, 128),
    tl.Mean(),
    tl.Dense(n_species),
    tl.Softmax(),
)

In [63]:
import trax 
from trax.supervised import training
from trax import layers as tl

# Training task.

X = np.zeros((len(encoded_sequences), 1024), dtype='int32')
Y = np.zeros((len(encoded_sequences), n_species), dtype='int32')


for i, s in enumerate(encoded_sequences):
    X[i, :len(s)] = (np.array(s) + 1)
    yi = species_indices[i]
    Y[i, yi] = 1
    
print(X.shape)

def batches():
    for i in range(len(X) // 4):
        yield (
            X[4*i:4*i+4, :], 
            Y[4*i:4*i+4, :])
        

batch_gen = batches()
eval_gen = batches()
for x, y in batch_gen:
    break
print(x.shape)
print(y.shape)


train_task = training.TrainTask(
    labeled_data=batch_gen,
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.01),
    n_steps_per_checkpoint=500,
)

eval_task = training.EvalTask(
    labeled_data=eval_gen,
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
    n_eval_batches=20  # For less variance in eval numbers.
)


(14993, 1024)
(4, 1024)
(4, 33)


In [64]:
training_loop = training.Loop(transformer, train_task, eval_tasks=[eval_task])
training_loop.run()

LayerError: Exception passing through layer Serial (in init):
  layer created in file [...]/trax/supervised/training.py, line 146
  layer input shapes: (ShapeDtype{shape:(4, 1024), dtype:int32}, ShapeDtype{shape:(4, 33), dtype:int32})

  File [...]/trax/layers/combinators.py, line 104, in init_weights_and_state
    sublayer.init(inputs, use_cache=True))

LayerError: Exception passing through layer CrossEntropyLoss (in init):
  layer created in file [...]/trax/layers/metrics.py, line 96
  layer input shapes: (ShapeDtype{shape:(4, 33), dtype:float32}, ShapeDtype{shape:(4, 33), dtype:int32})

  File [...]/trax/layers/combinators.py, line 105, in init_weights_and_state
    outputs, _ = sublayer._forward_abstract(inputs)

LayerError: Exception passing through layer _CrossEntropy (in _forward_abstract):
  layer created in file [...]/trax/layers/base.py, line 704
  layer input shapes: (ShapeDtype{shape:(4, 33), dtype:float32}, ShapeDtype{shape:(4, 33), dtype:int32})

  File [...]/jax/interpre